In [ ]:
from QHT import QHT
from exactqcnn_tensornet import ExactQcnnMPS
from my_tensornetwork import MyFiniteMPS

import numpy as np
import math
import os
import re
import json
from tqdm.notebook import tqdm

In [ ]:
class QHTMPS(QHT):
    def __init__(self, exactqcnn:ExactQcnnMPS):
        self.n_qubits = exactqcnn.n_qubits
        self.exactqcnn = exactqcnn

        # train1
        read_data = self._dataset_Pollmann("train1")
        self.h1h2_i_train1 = read_data[0]
        self.label_i_train1 = read_data[1] 
        # train2
        read_data = self._dataset_Pollmann("train2")
        self.h1h2_i_train2 = read_data[0]
        self.label_i_train2 = read_data[1] 
        # test1
        read_data = self._dataset_Pollmann("test1")
        self.h1h2_i_test1 = read_data[0]
        self.label_i_test1 = read_data[1]
        # test2
        read_data = self._dataset_Pollmann("test2")
        self.h1h2_i_test2 = read_data[0]
        self.label_i_test2 = read_data[1]


    def calc_error_separable_OP(self, n, alpha_min_li=[0.25], read_json=True, data="test1", mode="bayes"):
        folder_dmrg = "./tensornetPollmann/results"

        if data=="test1":
            J1J2_i = self.h1h2_i_test1
            label_i = self.label_i_test1
        elif data=="test2":
            J1J2_i = self.h1h2_i_test2
            label_i = self.label_i_test2

        if not read_json:
            raise Exception(f"Please calculate OP_Pollmann in {folder_dmrg}.")
        else:
            if data=="test1":
                # with open(f"./json_data/OP(L={self.n_qubits},{data})_ave_Z.json") as f:
                #     json_dict = json.load(f)
                # J1J2_json = json_dict["J1J2"]
                # ave_OP_json = json_dict["ave_Z"]
                # ave_OP_nt = np.mean(np.abs(np.array(ave_OP_json))[label_i==2])
                # ave_OP_tri = np.mean(np.abs(np.array(ave_OP_json))[label_i==0])
                return self.calc_error_separable_OP_FMz(n, alpha_min_li, read_json, data, mode)
            elif data=="test2":
                with open(f"./json_data/OP(L={self.n_qubits},{data})_ave_SOP.json") as f:
                    json_dict = json.load(f)
                J1J2_json = json_dict["J1J2"]
                ave_OP_json = json_dict["ave_SOP"]
                ave_OP_nt = np.mean(np.array(ave_OP_json)[label_i==1])
                ave_OP_tri = np.mean(np.array(ave_OP_json)[label_i==0])
            assert np.allclose(np.array(J1J2_json), J1J2_i)    

        bernoulli_p_nt = (ave_OP_nt+1)/2
        bernoulli_p_tri = (ave_OP_tri+1)/2

        # 尤度比検定: 手書きメモ参照
        nCr = math.comb
        n_is_int = type(n) == int
        if n_is_int:
            n_li = [n]
        else:
            n_li = n
        result_li = []
        for n in n_li:
            # k_alphaを見つけ、gammaも求める
            k_alpha_li = []
            gamma_li = []
            for alpha_min in alpha_min_li:
                F = 0
                for k in range(n+1):
                    f = nCr(n, k) * 0.5**n
                    F += f
                    if alpha_min >= 1-F:
                        k_alpha_li.append(k)
                        gamma_li.append((alpha_min-1+F)/f)
                        break
                    elif k==n:
                        k_alpha_li.append(k)
                        gamma_li.append((alpha_min-1+F)/f)
            # alphaとbetaを求める
            alpha_li = []
            beta_li = []
            for i, k_alpha in enumerate(k_alpha_li): 
                alpha = 1- sum([nCr(n,j)*bernoulli_p_tri**j*(1-bernoulli_p_tri)**(n-j) for j in range(k_alpha+1)])
                alpha += gamma_li[i] * nCr(n,k_alpha)*bernoulli_p_tri**k_alpha*(1-bernoulli_p_tri)**(n-k_alpha)
                alpha_li.append(alpha)
                beta = sum([nCr(n,j)*bernoulli_p_nt**j*(1-bernoulli_p_nt)**(n-j) for j in range(k_alpha)])
                beta += (1-gamma_li[i]) * nCr(n,k_alpha)*bernoulli_p_nt**k_alpha*(1-bernoulli_p_nt)**(n-k_alpha)
                beta_li.append(beta)
            result_li.append((alpha_li, beta_li))
        if n_is_int:
            return result_li[0]
        else:
            return result_li
        
    def gen_qNeyman(self, seed=None, a=0, a_li=None, shots=20*300*2, n_ent=3, data="train1", haar="haar"):
        self.shots = shots
        assert self.shots%20==0

        if data=="train1":
            J1J2_i = self.h1h2_i_train1
            label_i = self.label_i_train1
        elif data=="train2":
            J1J2_i = self.h1h2_i_train2
            label_i = self.label_i_train2

        folder = f"./random_unitary/random_{haar}/"
        folder_dmrg = "./tensornetPollmann/results"
        
        rng = np.random.default_rng(seed)
        file_idx = 1
        remained_random = np.load(folder+f"{n_ent}qubits_40x2x10000num({file_idx}).npy")
        remained_random2 = None
        rho_hat_li_nt = [0 for _ in range(0,self.n_qubits,n_ent)]
        rho_hat_li_tri = [0 for _ in range(0,self.n_qubits,n_ent)]
        
        for npz_path in tqdm(os.listdir(folder_dmrg+f"/dmrg_Pollmann_{data}/L={self.n_qubits}"), leave=False):
            match = re.search(r'J1=(-?\d+\.?\d*e?-?\d*),J2=(-?\d+\.?\d*e?-?\d*)', npz_path)
            J1J2 = [float(match.group(1)), float(match.group(2))]
            match = np.isclose(J1J2_i,J1J2)
            match = np.where(match[:,0]&match[:,1])[0]
            if match.shape[0] == 0:
                continue
            else:
                label = label_i[match[0]]
            for _ in tqdm(range(self.shots//20), leave=False):
                mps = MyFiniteMPS([arr.astype(np.complex128) for arr in list(np.load(folder_dmrg+f"/dmrg_Pollmann_{data}/L={self.n_qubits}/{npz_path}").values())])
                for i_q, qidx in enumerate(range(0,self.n_qubits,n_ent)):
                    if qidx+n_ent <= self.n_qubits:
                        _n = n_ent
                        while remained_random.shape[0] < 1:
                            file_idx += 1
                            try:
                                remained_random = np.concatenate([remained_random, np.load(folder+f"{n_ent}qubits_40x2x10000num({file_idx}).npy")])
                            except FileNotFoundError:
                                file_idx = 1
                                remained_random = np.concatenate([remained_random, np.load(folder+f"{n_ent}qubits_40x2x10000num({file_idx}).npy")])
                        choice_haar = remained_random[0]
                        remained_random = remained_random[1:]
                    else:
                        _n = self.n_qubits-qidx
                        file_idx2 = 1
                        if remained_random2 is None:
                            remained_random2 = np.load(folder+f"{_n}qubits_40x2x10000num({file_idx2}).npy")
                        while remained_random2.shape[0] < 1:
                            file_idx2 += 1
                            try:
                                remained_random2 = np.concatenate([remained_random2, np.load(folder+f"{_n}qubits_40x2x10000num({file_idx2}).npy")])
                            except FileNotFoundError:
                                file_idx2 = 1
                                remained_random2 = np.concatenate([remained_random2, np.load(folder+f"{_n}qubits_40x2x10000num({file_idx2}).npy")])
                        choice_haar = remained_random2[0]
                        remained_random2 = remained_random2[1:]
                    if _n==3:
                        meas_li = []
                        for i in range(2**_n):
                            op = choice_haar.T.conj()[:,i][:,None]@choice_haar[i][None,:]
                            meas_li.append(mps.measure_three_site_observable(op.reshape([2 for _ in range(int(2*_n))]), qidx,qidx+1,qidx+2).real)
                    elif _n==2:
                        meas_li = []
                        for i in range(2**_n):
                            op = choice_haar.T.conj()[:,i][:,None]@choice_haar[i][None,:]
                            meas_li.append(mps.measure_two_site_observable(op.reshape([2 for _ in range(int(2*_n))]), qidx,qidx+1).real)
                    elif _n==1:
                        meas_li = []
                        for i in range(2**_n):
                            op = choice_haar.T.conj()[:,i][:,None]@choice_haar[i][None,:]
                            meas_li.append(mps.measure_local_operator([op.reshape([2 for _ in range(int(2*_n))])], [qidx])[0].real)
                    else:
                        raise Exception("It has not supported yet!")
                    meas_int = rng.choice(len(meas_li), p=meas_li)
                    snapshot = (2**_n+1) * choice_haar.T.conj()[:,meas_int][:,None]@choice_haar[meas_int][None,:] - np.eye(2**_n)
                    if label==0:
                        rho_hat_li_tri[i_q] += snapshot
                    else:
                        rho_hat_li_nt[i_q] += snapshot
        for i, qidx in enumerate(range(0,self.n_qubits,n_ent)):
            rho_hat_li_tri[i] /= np.count_nonzero(label_i==0)
            rho_hat_li_nt[i] /= np.count_nonzero(label_i!=0)

        if a_li is None:
            vv_li = []
            uu_li = []
            for rho_nt, rho_tri in zip(rho_hat_li_nt, rho_hat_li_tri):
                vv, uu = np.linalg.eigh(rho_tri - np.exp(a)*rho_nt)
                vv_li.append(vv)
                uu_li.append(uu)
            self.qNeyman_hat = [[vv_li, uu_li]]
        else:
            self.qNeyman_hat = []
            for a in tqdm(a_li, leave=False):
                vv_li = []
                uu_li = []
                for rho_nt, rho_tri in zip(rho_hat_li_nt, rho_hat_li_tri):
                    vv, uu = np.linalg.eigh(rho_tri - np.exp(a)*rho_nt)
                    vv_li.append(vv)
                    uu_li.append(uu)
                self.qNeyman_hat.append([vv_li, uu_li])

        return 
    
    def multi_func1(self, multi_input):
        prob = 0
        meas_li = multi_input[1]
        multi_input = multi_input[0]
        for basis_idx in multi_input:
            basis_idx = format(basis_idx, '0' + str(self.n_qubits) + 'b')
            basis_idx = np.array([int(x) for x in basis_idx], dtype=bool)
            prob += np.prod(np.hstack([meas_li[~basis_idx], (1-meas_li[basis_idx])]))
        return prob
    def calc_error_separable_CSqNeyman(self, n, seed=None, a_li=[0], shots=20*300*2, 
                                       n_ent=3, data="test1", haar="haar",
                                       read_json=True, read_npz=True):
        folder_dmrg = "./tensornetPollmann/results"
        if data=="test1":
            J1J2_i = self.h1h2_i_test1
            label_i = self.label_i_test1
            data_train = "train1"
        elif data=="test2":
            J1J2_i = self.h1h2_i_test2
            label_i = self.label_i_test2
            data_train = "train2"
        elif data=="train1":
            J1J2_i = self.h1h2_i_train1
            label_i = self.label_i_train1
            data_train = "train1"

        if not read_json:
            if read_npz:
                folder_npz = f"./npz_data/CSqNeyman(L={self.n_qubits}_shots={shots}_seed={seed}_nent={n_ent})/"
                assert (np.array(a_li) == np.load(folder_npz+f"CSqNeyman(L={self.n_qubits}_shots={shots}_seed={seed}_nent={n_ent},{data_train}{','+haar if haar!='haar' else ''})_a_li.npy")).all()
                qNeyman_hat = []
                vv_li_li = np.load(folder_npz+f"CSqNeyman(L={self.n_qubits}_shots={shots}_seed={seed}_nent={n_ent},{data_train}{','+haar if haar!='haar' else ''})_vv_li.npz").values()
                uu_li_li = np.load(folder_npz+f"CSqNeyman(L={self.n_qubits}_shots={shots}_seed={seed}_nent={n_ent},{data_train}{','+haar if haar!='haar' else ''})_uu_li.npz").values()
                num_dev = math.ceil(self.n_qubits/n_ent)
                vv_li_li = [x for x in vv_li_li]
                vv_li_li = [vv_li_li[i:i+num_dev] for i in range(0,len(vv_li_li),num_dev)]
                uu_li_li = [x for x in uu_li_li]
                uu_li_li = [uu_li_li[i:i+num_dev] for i in range(0,len(uu_li_li),num_dev)]
                for vv_li, uu_li in zip(vv_li_li, uu_li_li):
                    qNeyman_hat.append([vv_li, uu_li])
                self.qNeyman_hat = qNeyman_hat
                qNeyman_hat=None; vv_li_li=None; uu_li_li=None
            else:
                self.gen_qNeyman(seed, a_li=a_li, shots=shots, n_ent=n_ent, data=data_train, haar=haar)
                folder_npz = f"./CSqNeyman(L={self.n_qubits}_shots={shots}_seed={seed}_nent={n_ent})/"
                os.makedirs(folder_npz)
                np.save(folder_npz+f"CSqNeyman(L={self.n_qubits}_shots={shots}_seed={seed}_nent={n_ent},{data_train}{','+haar if haar!='haar' else ''})_a_li.npy", a_li)
                np.savez_compressed(folder_npz+f"CSqNeyman(L={self.n_qubits}_shots={shots}_seed={seed}_nent={n_ent},{data_train}{','+haar if haar!='haar' else ''})_vv_li.npz", *sum([x[0] for x in self.qNeyman_hat],[]))
                np.savez_compressed(folder_npz+f"CSqNeyman(L={self.n_qubits}_shots={shots}_seed={seed}_nent={n_ent},{data_train}{','+haar if haar!='haar' else ''})_uu_li.npz", *sum([x[1] for x in self.qNeyman_hat],[]))

            rng = np.random.default_rng()
            alpha_li = [0 for _ in range(len(a_li))]
            beta_li = [0 for _ in range(len(a_li))]
            for npz_path in tqdm(os.listdir(folder_dmrg+f"/dmrg_Pollmann_{data}/L={self.n_qubits}"), leave=False):
                match = re.search(r'J1=(-?\d+\.?\d*e?-?\d*),J2=(-?\d+\.?\d*e?-?\d*)', npz_path)
                J1J2 = [float(match.group(1)), float(match.group(2))]
                match = np.isclose(J1J2_i,J1J2)
                match = np.where(match[:,0]&match[:,1])[0]
                if match.shape[0] == 0:
                    continue
                else:
                    label = label_i[match[0]]
                for i_a, a in enumerate(a_li):
                    if a!=0:
                        continue
                    mps = MyFiniteMPS([arr.astype(np.complex128) for arr in list(np.load(folder_dmrg+f"/dmrg_Pollmann_{data}/L={self.n_qubits}/{npz_path}").values())])
                    meas_li = []
                    qidx = 0
                    for uu in self.qNeyman_hat[i_a][1]:
                        if uu.shape[0]==2**3:
                            li = []
                            for i in range(2**3):
                                op = uu[:,i][:,None]@uu.T.conj()[i][None,:]
                                li.append(mps.measure_three_site_observable(op.reshape([2 for _ in range(int(2*3))]), qidx,qidx+1,qidx+2).real)
                            meas_li.append(li)
                            qidx += 3
                        elif uu.shape[0]==2**2:
                            li = []
                            for i in range(2**2):
                                op = uu[:,i][:,None]@uu.T.conj()[i][None,:]
                                li.append(mps.measure_two_site_observable(op.reshape([2 for _ in range(int(2*2))]), qidx,qidx+1).real)
                            meas_li.append(li)
                            qidx += 2  
                        elif uu.shape[0]==2**1:
                            li = []
                            for i in range(2**1):
                                op = uu[:,i][:,None]@uu.T.conj()[i][None,:]
                                li.append(mps.measure_local_operator([op.reshape([2 for _ in range(int(2*1))])], [qidx])[0].real)
                            meas_li.append(li)
                            qidx += 1   
                        else:
                            raise Exception("It has not supported yet!")
                    # #multicpt
                    # num_div = len(self.qNeyman_hat[i][1])//(cpu_count()-1)
                    # with Pool(cpu_count()-1) as p:
                    #     multi_output_li = p.map(self.multi_func1, [(self.qNeyman_hat[i][1][j:j+num_div], meas_li) for j in range(0,len(self.qNeyman_hat[i][1]),num_div)])
                    # prob = sum(multi_output_li)
                    # #singlecpu
                    # prob = 0
                    # for basis_idx in self.qNeyman_hat[i][1]:
                    #     basis_idx = format(basis_idx, '0' + str(self.n_qubits) + 'b')
                    #     basis_idx = np.array([int(x) for x in basis_idx], dtype=bool)
                    #     prob += np.prod(np.hstack([meas_li[~basis_idx], (1-meas_li[basis_idx])]))
                    max_length = max(len(sublist) for sublist in meas_li) # zero padding (for transforming list to array)
                    meas_li = np.array([sublist + [0] * (max_length - len(sublist)) for sublist in meas_li])
                    max_length = max(len(sublist) for sublist in self.qNeyman_hat[i_a][0])
                    vv_li = np.array([sublist.tolist() + [0] * (max_length - len(sublist)) for sublist in self.qNeyman_hat[i_a][0]])
                    K = int(1e+7)
                    sample_li = np.array([rng.choice(meas_li.shape[1], size=K, p=probabilities) for probabilities in meas_li])
                    selected_values = vv_li[np.arange(meas_li.shape[0])[:, None], sample_li]
                    # product_values = np.prod(selected_values, axis=0)
                    # positive_prob = np.count_nonzero(product_values > 0)/K
                    positive_count = np.sum(selected_values>0, axis=0)
                    positive_prob = np.count_nonzero(positive_count > meas_li.shape[0]//2)/K
                    if label==0:
                        alpha_li[i_a] += positive_prob
                    else:
                        beta_li[i_a] += positive_prob
            alpha_li = 1 - np.array(alpha_li)/np.count_nonzero(label_i==0)
            beta_li = np.array(beta_li)/np.count_nonzero(label_i!=0)
            
            self.alpha_li = alpha_li; self.beta_li = beta_li
            with open(f"./CSqNeyman(L={self.n_qubits}_shots={shots}_nent={n_ent},{data}{','+haar if haar!='haar' else ''})_alphabeta.json", "w") as f:
                json.dump({"a_li":a_li, "alpha":alpha_li.tolist(), "beta":beta_li.tolist()}, f, indent=2)

        else:
            with open(f"./json_data/CSqNeyman(L={self.n_qubits}_shots={shots}_nent={n_ent},{data}{','+haar if haar!='haar' else ''})_alphabeta.json") as f:
                json_dict = json.load(f)
            assert (np.array(a_li) == np.array(json_dict["a_li"])).all()
            alpha_li = np.array(json_dict["alpha"])
            beta_li = np.array(json_dict["beta"])

        n_is_int = type(n) == int
        if n_is_int:
            n_li = [n]
        else:
            n_li = n
        result_li = []
        for n in n_li:
            if n>1:
                nCr = math.comb
                # (1-alpha)よりalphaの方を多く得る確率=alpha_n
                alpha_n_li = np.sum([nCr(n, r)*alpha_li**(n-r)*(1-alpha_li)**r for r in range(n//2+1)], axis=0)
                beta_n_li = np.sum([nCr(n, r)*beta_li**(n-r)*(1-beta_li)**r for r in range(n//2+1)], axis=0)
                # 古典Neyman-Pearsonでいうgamma=0.5なので、足し過ぎた分引く
                if n%2 == 0:
                    alpha_n_li -= 0.5 * nCr(n,n//2)*alpha_li**(n//2)*(1-alpha_li)**(n//2)
                    beta_n_li -= 0.5 * nCr(n,n//2)*beta_li**(n//2)*(1-beta_li)**(n//2)
            elif n==1:
                alpha_n_li = alpha_li; beta_n_li = beta_li
            result_li.append((alpha_n_li.tolist(), beta_n_li.tolist()))
        if n_is_int:
            return result_li[0]
        else:
            return result_li 
        
    def calc_error_separable_qcnn(self, n, shots_per_rho=None, index=150,
                                  alpha_min_li=[0.25],
                                  read_json=True, data="test1",
                                  qcnn_mode="qcnn1"):
        if data=="test1":
            J1J2_i = self.h1h2_i_test1
            label_i = self.label_i_test1
        elif data=="test2":
            J1J2_i = self.h1h2_i_test2
            label_i = self.label_i_test2

        if not read_json:
            raise Exception(f"Please calculate qcnn_Pollmann in this directory.")
        else:
            with open(f"./json_data/{qcnn_mode}(L={self.n_qubits}_shots={shots_per_rho},{data}).json") as f:
                json_dict = json.load(f)
            ave_qcnn_json = json_dict["expectation_list"][f"index={index}"]
            ave_qcnn_nt = np.mean(np.array(ave_qcnn_json)[label_i!=0])
            ave_qcnn_tri = np.mean(np.array(ave_qcnn_json)[label_i==0])

        if "qcnn1" in qcnn_mode:
            p0 = 3/4
        elif "qcnn2" in qcnn_mode:
            p0 = 1/2

        bernoulli_p_nt = (ave_qcnn_nt+1)/2
        bernoulli_p_tri = (ave_qcnn_tri+1)/2

        # 尤度比検定: 手書きメモ参照
        nCr = math.comb
        n_is_int = type(n) == int
        if n_is_int:
            n_li = [n]
        else:
            n_li = n
        result_li = []
        for n in n_li:
            # k_alphaを見つけ、gammaも求める
            k_alpha_li = []
            gamma_li = []
            for alpha_min in alpha_min_li:
                F = 0
                for k in range(n+1):
                    f = nCr(n, k) * p0**k*(1-p0)**(n-k)
                    F += f
                    if alpha_min >= 1-F:
                        k_alpha_li.append(k)
                        gamma_li.append((alpha_min-1+F)/f)
                        break
                    elif k==n:
                        k_alpha_li.append(k)
                        gamma_li.append((alpha_min-1+F)/f)
            # alphaとbetaを求める
            alpha_li = []
            beta_li = []
            for i, k_alpha in enumerate(k_alpha_li): 
                alpha = 1- sum([nCr(n,j)*bernoulli_p_tri**j*(1-bernoulli_p_tri)**(n-j) for j in range(k_alpha+1)])
                alpha += gamma_li[i] * nCr(n,k_alpha)*bernoulli_p_tri**k_alpha*(1-bernoulli_p_tri)**(n-k_alpha)
                alpha_li.append(alpha)
                beta = sum([nCr(n,j)*bernoulli_p_nt**j*(1-bernoulli_p_nt)**(n-j) for j in range(k_alpha)])
                beta += (1-gamma_li[i]) * nCr(n,k_alpha)*bernoulli_p_nt**k_alpha*(1-bernoulli_p_nt)**(n-k_alpha)
                beta_li.append(beta)
            result_li.append((alpha_li, beta_li))
        if n_is_int:
            return result_li[0]
        else:
            return result_li
        
    def calc_error_separable_exactqcnn(self, n, alpha_min_li=[0.25], read_json=True, data="test1"):
        folder_dmrg = "./tensornetPollmann/results"

        if data=="train1":
            J1J2_i = self.h1h2_i_train1
            label_i = self.label_i_train1 
        elif data=="train2":
            J1J2_i = self.h1h2_i_train2
            label_i = self.label_i_train2  
        elif data=="test1":
            J1J2_i = self.h1h2_i_test1
            label_i = self.label_i_test1
        elif data=="test2":
            J1J2_i = self.h1h2_i_test2
            label_i = self.label_i_test2

        if not read_json:
            meas_li = [None for _ in range(len(J1J2_i))]
            for npz_path in tqdm(os.listdir(folder_dmrg+f"/dmrg_Pollmann_{data}/L={self.n_qubits}"), leave=False):
                match = re.search(r'J1=(-?\d+\.?\d*e?-?\d*),J2=(-?\d+\.?\d*e?-?\d*)', npz_path)
                J1J2 = [float(match.group(1)), float(match.group(2))]
                match = np.isclose(J1J2_i,J1J2)
                match = np.where(match[:,0]&match[:,1])[0]
                if match.shape[0] == 0:
                    continue
                else:
                    label = label_i[match[0]]
                mps = MyFiniteMPS(list(np.load(folder_dmrg+f"/dmrg_Pollmann_{data}/L={self.n_qubits}/{npz_path}").values()))
                if data=="test1" or data=="train1":
                    meas = self.exactqcnn.meas_Z(mps) # exactqcnnにcomplexが無いので,mpsもfloatで良い
                elif data=="test2" or data=="train2":
                    meas = self.exactqcnn.meas_SOP(mps) # exactqcnnにcomplexが無いので,mpsもfloatで良い
                meas_li[match[0]] = meas
            if data=="test1" or data=="train1":
                with open(f"./exactqcnn(L={self.n_qubits},{data})_ave_Z.json", "w") as f:
                    json.dump({"J1J2":np.array(J1J2_i).tolist(), "ave_Z":meas_li}, f, indent=2)
            elif data=="test2" or data=="train2":
                with open(f"./exactqcnn(L={self.n_qubits},{data})_ave_SOP.json", "w") as f:
                    json.dump({"J1J2":np.array(J1J2_i).tolist(), "ave_SOP":meas_li}, f, indent=2)
            ave_exactqcnn_nt = np.mean(np.array(meas_li)[label_i!=0])
            ave_exactqcnn_tri = np.mean(np.array(meas_li)[label_i==0])
        else:
            if data=="test1" or data=="train1":
                with open(f"./json_data/exactqcnn(L={self.n_qubits},{data})_ave_Z.json") as f:
                    json_dict = json.load(f)
                J1J2_json = json_dict["J1J2"]
                ave_exactqcnn_json = json_dict["ave_Z"]
                ave_exactqcnn_nt = np.mean(np.array(ave_exactqcnn_json)[label_i!=0])
                ave_exactqcnn_tri = np.mean(np.array(ave_exactqcnn_json)[label_i==0])
            elif data=="test2" or data=="train2":
                with open(f"./json_data/exactqcnn(L={self.n_qubits},{data})_ave_SOP.json") as f:
                    json_dict = json.load(f)
                J1J2_json = json_dict["J1J2"]
                ave_exactqcnn_json = json_dict["ave_SOP"]
                ave_exactqcnn_nt = np.mean(np.array(ave_exactqcnn_json)[label_i!=0])
                ave_exactqcnn_tri = np.mean(np.array(ave_exactqcnn_json)[label_i==0])
            assert np.allclose(np.array(J1J2_json), J1J2_i)

        p0 = 3/4

        bernoulli_p_nt = (ave_exactqcnn_nt+1)/2
        bernoulli_p_tri = (ave_exactqcnn_tri+1)/2

        # 尤度比検定: 手書きメモ参照
        nCr = math.comb
        n_is_int = type(n) == int
        if n_is_int:
            n_li = [n]
        else:
            n_li = n
        result_li = []
        for n in n_li:
            # k_alphaを見つけ、gammaも求める
            k_alpha_li = []
            gamma_li = []
            for alpha_min in alpha_min_li:
                F = 0
                for k in range(n+1):
                    f = nCr(n, k) * p0**k*(1-p0)**(n-k)
                    F += f
                    if alpha_min >= 1-F:
                        k_alpha_li.append(k)
                        gamma_li.append((alpha_min-1+F)/f)
                        break
                    elif k==n:
                        k_alpha_li.append(k)
                        gamma_li.append((alpha_min-1+F)/f)
            # alphaとbetaを求める
            alpha_li = []
            beta_li = []
            for i, k_alpha in enumerate(k_alpha_li): 
                alpha = 1- sum([nCr(n,j)*bernoulli_p_tri**j*(1-bernoulli_p_tri)**(n-j) for j in range(k_alpha+1)])
                alpha += gamma_li[i] * nCr(n,k_alpha)*bernoulli_p_tri**k_alpha*(1-bernoulli_p_tri)**(n-k_alpha)
                alpha_li.append(alpha)
                beta = sum([nCr(n,j)*bernoulli_p_nt**j*(1-bernoulli_p_nt)**(n-j) for j in range(k_alpha)])
                beta += (1-gamma_li[i]) * nCr(n,k_alpha)*bernoulli_p_nt**k_alpha*(1-bernoulli_p_nt)**(n-k_alpha)
                beta_li.append(beta)
            result_li.append((alpha_li, beta_li))
        if n_is_int:
            return result_li[0]
        else:
            return result_li


In [ ]:
exactqcnn = ExactQcnnMPS(27, max_singular_values=200, max_truncation_err2=2e-5)
ins = QHTMPS(exactqcnn)

In [ ]:
ins.calc_error_separable_CSqNeyman(n=1, seed=2, 
                                   a_li=[i/10 for i in range(-10,11)],shots=20*60*2, 
                                   n_ent=3, data="test2", haar="haar",
                                   read_json=False,read_npz=True)

## Plot

In [ ]:
class PlotData:
    def __init__(self, qht:QHTMPS):
        self.qht = qht
        self.results_list_alpha_vs_beta = []
        self.results_list_n_vs_logbeta = []
        self.results_list_beta_vs_n = []
        self.results_list_shots_vs_n = [] 
    def get_results_alpha_vs_beta(self, n, data="test1"):
        results_error_li = []
        results_error_point = []
        settings_dict = []
        results_error_li.append(self.qht.calc_error_separable_OP(n, alpha_min_li=[i/100 for i in range(1,100)],
                                                                 read_json=True, data=data))
        settings_dict.append({"color":"b","linestyle":"-","label":"cNeyman(OP)"})
        results_error_li.append(self.qht.calc_error_separable_CSqNeyman(n, seed=2, 
                                a_li=[i/10 for i in range(-10,11)],shots=20*15*2, 
                                n_ent=3, data=data,
                                read_json=True,read_npz=False))
        settings_dict.append({"color":"brown","marker":".","linestyle":"--","label":"tomography"})
        results_error_li.append(self.qht.calc_error_separable_qcnn(n, shots_per_rho=None,
                                                                   alpha_min_li=[i/100 for i in range(1,100)],
                                                                   index=150, read_json=True, data=data,
                                                                   qcnn_mode="qcnn1"))
        settings_dict.append({"color":"m","linestyle":"-","label":"qcnn"})
        results_error_li.append(self.qht.calc_error_separable_exactqcnn(n, alpha_min_li=[i/100 for i in range(1,100)], read_json=True, data=data))
        settings_dict.append({"color":"g","linestyle":"-","label":"exactqcnn"})
        self.results_list_alpha_vs_beta.append([n, results_error_li, results_error_point, settings_dict])
    def get_results_beta_vs_n(self, n_max, data="test1"):
        n_li = list(range(1,n_max+1))
        results_error_li = []
        results_error_point = []
        settings_dict = []
        results_error_li.append(self.qht.calc_error_separable_OP(n_li, alpha_min_li=[i/500 for i in range(1,500)],
                                                                 read_json=True, data=data))
        settings_dict.append({"color":"b", "linestyle":"", "marker":".", "label":"cNeyman(OP)"})
        results_error_li.append(self.qht.calc_error_separable_CSqNeyman(n_li, seed=2, 
                                a_li=[i/10 for i in range(-10,11)],shots=20*15*2,
                                n_ent=3, data=data,
                                read_json=True,read_npz=True))
        settings_dict.append({"color":"brown", "linestyle":"", "marker":".", "label":"tomography"})
        results_error_li.append(self.qht.calc_error_separable_qcnn(n_li, shots_per_rho=None, 
                                                                   alpha_min_li=[i/500 for i in range(1,500)],
                                                                   index=150, read_json=True, data=data,
                                                                   qcnn_mode="qcnn1"))
        settings_dict.append({"color":"m", "linestyle":"", "marker":".", "label":"qcnn"})
        results_error_li.append(self.qht.calc_error_separable_exactqcnn(n_li, alpha_min_li=[i/500 for i in range(1,500)], read_json=True, data=data))
        settings_dict.append({"color":"g", "linestyle":"", "marker":".", "label":"exactqcnn"})
        self.results_list_beta_vs_n.append([results_error_li, results_error_point, settings_dict])
    def get_results_shots_vs_n(self, n_max, data="test1"):
        n_li = list(range(1,n_max+1))
        shots_li = []
        results_error_li = []
        results_error_point = []
        settings_dict = []
        shots_li.append([320,600,1200,2400])
        results_error_li.append([self.qht.calc_error_separable_CSqNeyman(n_li, seed=2, 
                                a_li=[i/10 for i in range(-10,11)], shots=shots,
                                n_ent=2, data=data,
                                read_json=True,read_npz=True) for shots in shots_li[-1]])
        settings_dict.append({"color":"brown", "linestyle":"--", "marker":"o", "label":"tomography"})
        # shots_per_rho = 500
        # shots_li.append([i*20*shots_per_rho*2 for i in [150]])
        # results_error_point.append([self.qht.calc_error_separable_qcnn(n_li, shots_per_rho=shots_per_rho, 
        #                                                               index=shots//(20*shots_per_rho*2), read_json=True, data=data,
        #                                                               qcnn_mode="qcnn2") for shots in shots_li[-1]])                                                    
        # settings_dict.append({"color":"m", "linestyle":"--", "marker":"o", "label":"qcnn"})
        self.results_list_shots_vs_n.append([shots_li, results_error_li, results_error_point, settings_dict])

ins_plotdata = PlotData(qht=ins)

##### $\beta_n$ vs. $\alpha_n$

In [5]:
ins_plotdata.get_results_alpha_vs_beta(1, "test2")

In [ ]:
ins.plot_alpha_vs_beta(*ins_plotdata.results_list_alpha_vs_beta[0])

##### $n$ vs. $\beta_n$

In [11]:
ins_plotdata.get_results_beta_vs_n(20, "test2")

In [ ]:
for alpha in [0.3,0.1,0.05]:
    ins.plot_beta_vs_n(alpha, *ins_plotdata.results_list_beta_vs_n[0])

##### $n$ vs. training shots

In [7]:
ins_plotdata.get_results_shots_vs_n(50, "test1")

In [ ]:
for a in [0.3,0.1,0.05]:
    ins.plot_shots_vs_n(a, a, *ins_plotdata.results_list_shots_vs_n[0])